# Лабораторная работа №1 (Зябрев Кирилл, МО-221)

# Гипотезы:

1) Окупаемость фильма тем больше, чем меньше возрастной рейтинг фильма.
2) Фильмы с элементами драмы чаще всех имеют IMDB рейтинг выше 8. 
3) Фильмы с бюджетом выше среднего (по миру) имеют в среднем более высокую продолжительность, чем остальные.

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np

Создадим датафрейм

In [2]:
df = pd.read_csv("..\data\Dataset1.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   color                       5025 non-null   object 
 1   Director_Name               4872 non-null   object 
 2   num_Critic_for_reviews      4927 non-null   float64
 3   duration                    4959 non-null   float64
 4   director_Facebook_likes     4872 non-null   float64
 5   actor_3_Facebook_likes      4953 non-null   float64
 6   actor_2_name                4963 non-null   object 
 7   Actor_1_Facebook_likes      4968 non-null   float64
 8   gross                       4104 non-null   float64
 9   genres                      4974 non-null   object 
 10  actor_1_name                4968 non-null   object 
 11  movie_Title                 4974 non-null   object 
 12  num_voted_users             4974 non-null   float64
 13   cast_total_facebook_likes  4974 

Форматируем названия столбцов датафрейма:

In [3]:
df = df.rename (
    columns = {
        'Director_Name' : 'director_name',
        'num_Critic_for_reviews' : 'num_critic_for_reviews',
        'director_Facebook_likes' : 'director_facebook_likes',
        'actor_3_Facebook_likes' : 'actor_3_facebook_likes',
        'Actor_1_Facebook_likes' : 'actor_1_facebook_likes',
        ' cast_total_facebook_likes' : 'cast_total_facebook_likes',
        'movie_Title' : 'movie_title',
        'movie_facebook_likes;' : 'movie_facebook_likes'
    }
)

df.columns          

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

Найдем число явных дубликатов:

In [4]:
df.duplicated().sum()

45

Избавимся от них:

In [5]:
df = df.drop_duplicates().reset_index(drop = True)

Проверим:

In [6]:
df.duplicated().sum()

0

Заметим, что некоторые данные внесены в датасет некорректно.

In [7]:
df['color'].unique()

array(['Color', 'Colour', nan, 'color',
       'Color,Andrew Adamson,284,150,80,82,Kiran Shah,1000,291709845,Adventure|Family|Fantasy,Jim Broadbent,"The Chronicles of Narnia: The Lion, the Witch and the Wardrobe\xa0",286506,1317,Shane Rangi,5,hide and seek|lion|magic|professor|snow,http://www.imdb.com/title/tt0363771/?ref_=fn_tt_tt_1,1463,English,USA,PG,180000000,2005,190,6.9,2.35,0;',
       ' Black and White',
       'Color,Pete Docter,250,92,0,773,John Ratzenberger,12000,289907418,Adventure|Animation|Comedy|Family|Fantasy,Steve Buscemi,"Monsters, Inc.\xa0",585659,15013,James Coburn,0,friend|little girl|monster|rival|scream,http://www.imdb.com/title/tt0198781/?ref_=fn_tt_tt_1,593,English,USA,G,115000000,2001,1000,8.1,1.85,0;',
       'Color,Alex Proyas,225,115,295,466,Bruce Greenwood,10000,144795350,Action|Mystery|Sci-Fi|Thriller,Will Smith,"I, Robot\xa0",387632,12068,Chi McBride,1,humanoid robot|man versus machine|prosthetic limb|robot as pathos|robot vs. robot,http://www.imdb.com/t

Некоторые элементы Series 'color' представляют собой определенное подмножество совокупной информации о фильме. Попробуем вернуть информацию в наш датасет, предварительно избавившись от NaN значений:

In [8]:
df['color'] = df['color'].fillna("unknown")

При взгляде на данные становится понятно, что любое число больше длины фразы Black and White будет представлять размер строки с потеряннными данными. Пусть оно будет равно 30.

In [9]:
lost_data = df['color'].unique().tolist()
var = []

for i in lost_data:
    if len(i) > 30:
        var.append(i)
        
print(var)

['Color,Andrew Adamson,284,150,80,82,Kiran Shah,1000,291709845,Adventure|Family|Fantasy,Jim Broadbent,"The Chronicles of Narnia: The Lion, the Witch and the Wardrobe\xa0",286506,1317,Shane Rangi,5,hide and seek|lion|magic|professor|snow,http://www.imdb.com/title/tt0363771/?ref_=fn_tt_tt_1,1463,English,USA,PG,180000000,2005,190,6.9,2.35,0;', 'Color,Pete Docter,250,92,0,773,John Ratzenberger,12000,289907418,Adventure|Animation|Comedy|Family|Fantasy,Steve Buscemi,"Monsters, Inc.\xa0",585659,15013,James Coburn,0,friend|little girl|monster|rival|scream,http://www.imdb.com/title/tt0198781/?ref_=fn_tt_tt_1,593,English,USA,G,115000000,2001,1000,8.1,1.85,0;', 'Color,Alex Proyas,225,115,295,466,Bruce Greenwood,10000,144795350,Action|Mystery|Sci-Fi|Thriller,Will Smith,"I, Robot\xa0",387632,12068,Chi McBride,1,humanoid robot|man versus machine|prosthetic limb|robot as pathos|robot vs. robot,http://www.imdb.com/title/tt0343818/?ref_=fn_tt_tt_1,789,English,USA,PG-13,120000000,2004,981,7.1,2.35,0;', 

Удалим добавленные значения 

In [10]:
df = df[df['color'].apply(lambda x: len(x) <= 30)]

Создадим содержимое нового csv файла, который будет основой для датасета из ошибочно записанных данных.

In [11]:
csv_content = ','.join(df.columns.to_list())
csv_content += ';\n'

count = 0

for i in var:
    count += 1
    csv_content += str(i)
    if count != len(var):
        csv_content += '\n'

print(csv_content)

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes;
Color,Andrew Adamson,284,150,80,82,Kiran Shah,1000,291709845,Adventure|Family|Fantasy,Jim Broadbent,"The Chronicles of Narnia: The Lion, the Witch and the Wardrobe ",286506,1317,Shane Rangi,5,hide and seek|lion|magic|professor|snow,http://www.imdb.com/title/tt0363771/?ref_=fn_tt_tt_1,1463,English,USA,PG,180000000,2005,190,6.9,2.35,0;
Color,Pete Docter,250,92,0,773,John Ratzenberger,12000,289907418,Adventure|Animation|Comedy|Family|Fantasy,Steve Buscemi,"Monsters, Inc. ",585659,15013,James Coburn,0,friend|little girl|monster|rival|scream,http://www.imdb.com/title/tt0198781/?r

Запишем содержимое в новый файл LostData.csv.

In [12]:
new_csv = open("..\data\LostData.csv", "w+", encoding="utf-8")
new_csv.write(csv_content)
new_csv.close()

Создадим новый датафрейм на основе файла.

In [13]:
new_df = pd.read_csv("..\data\LostData.csv")
new_df.head(10)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes;
0,Color,Andrew Adamson,284.0,150,80.0,82.0,Kiran Shah,1000.0,291709845.0,Adventure|Family|Fantasy,...,1463.0,English,USA,PG,180000000.0,2005.0,190.0,6.9,2.35,0;
1,Color,Pete Docter,250.0,92,0.0,773.0,John Ratzenberger,12000.0,289907418.0,Adventure|Animation|Comedy|Family|Fantasy,...,593.0,English,USA,G,115000000.0,2001.0,1000.0,8.1,1.85,0;
2,Color,Alex Proyas,225.0,115,295.0,466.0,Bruce Greenwood,10000.0,144795350.0,Action|Mystery|Sci-Fi|Thriller,...,789.0,English,USA,PG-13,120000000.0,2004.0,981.0,7.1,2.35,0;
3,NaN,Christopher Barnard,NaN,22,0.0,NaN,NaN,5.0,NaN,Comedy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.2,NaN,0;
4,Color,Stuart Beattie,308.0,92,72.0,568.0,Kevin Grevioux,623.0,19059018.0,Action|Fantasy|Sci-Fi|Thriller,...,194.0,English,Australia,PG-13,65000000.0,2014.0,593.0,5.1,2.35,15000;
5,Color,NaN,20.0,22,NaN,271.0,Soleil Moon Frye,870.0,NaN,Comedy|Family|Fantasy,...,94.0,English,USA,TV-G,3000000.0,NaN,558.0,6.6,1.33,990;
6,Color,Anthony Russo,136.0,110,94.0,240.0,Billy Gardell,277.0,75604320.0,Comedy|Romance,...,195.0,English,USA,PG-13,54000000.0,2006.0,245.0,5.6,1.85,0;
7,Color,Bobby Farrelly,155.0,116,101.0,79.0,Tony Cox,889.0,90567722.0,Comedy,...,481.0,English,USA,R,51000000.0,2000.0,624.0,6.5,1.85,0;
8,Color,Glenn Ficarra,310.0,118,43.0,7000.0,Emma Stone,33000.0,84244877.0,Comedy|Drama|Romance,...,292.0,English,USA,PG-13,50000000.0,2011.0,15000.0,7.4,2.39,44000;
9,Color,Melville Shavelson,8.0,111,5.0,559.0,Tom Bosley,6000.0,NaN,Comedy|Family,...,61.0,English,USA,Unrated,2500000.0,1968.0,584.0,7.2,1.85,0;


Сконкатенируем оба датасета.

In [14]:
df = pd.concat([df, new_df])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4998 entries, 0 to 67
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4997 non-null   object 
 1   director_name              4895 non-null   object 
 2   num_critic_for_reviews     4949 non-null   float64
 3   duration                   4983 non-null   float64
 4   director_facebook_likes    4895 non-null   float64
 5   actor_3_facebook_likes     4975 non-null   float64
 6   actor_2_name               4985 non-null   object 
 7   actor_1_facebook_likes     4991 non-null   float64
 8   gross                      4124 non-null   float64
 9   genres                     4998 non-null   object 
 10  actor_1_name               4991 non-null   object 
 11  movie_title                4998 non-null   object 
 12  num_voted_users            4998 non-null   float64
 13  cast_total_facebook_likes  4998 non-null   float64
 14 

Вычислим количество NaN значений в каждом столбце:

In [15]:
df.isna().sum()      

color                           1
director_name                 103
num_critic_for_reviews         49
duration                       15
director_facebook_likes       103
actor_3_facebook_likes         23
actor_2_name                   13
actor_1_facebook_likes          7
gross                         874
genres                          0
actor_1_name                    7
movie_title                     0
num_voted_users                 0
cast_total_facebook_likes       0
actor_3_name                   23
facenumber_in_poster           13
plot_keywords                 152
movie_imdb_link                 0
num_user_for_reviews           21
language                       14
country                         5
content_rating                301
budget                        487
title_year                    107
actor_2_facebook_likes         13
imdb_score                      0
aspect_ratio                  327
movie_facebook_likes           68
movie_facebook_likes;        4930
dtype: int64

Разделим NaN значения на числовые и строковые. Затем напишем функцию replace_value(), которая будет заменять пустые значения в соответствие с заданными списком заголовков и значением, и воспользуемся ей.

In [16]:
columns_to_replace_str = ['color', 'director_name', 'actor_2_name', 'genres', 'actor_1_name', 'movie_title', 'actor_3_name', 
                      'plot_keywords', 'movie_imdb_link', 'language', 'country', 'content_rating']

def replace_value(df, columns, value):
    for row in columns:
        df[row] = df[row].fillna(value)
    return df
    
df = replace_value(df, columns_to_replace_str, 'unknown')

In [17]:
temp = df.columns.values.tolist()
columns_to_replace_num = []

for column in temp:
    if column not in columns_to_replace_str:
        columns_to_replace_num.append(column)

print(columns_to_replace_num)

['num_critic_for_reviews', 'duration', 'director_facebook_likes', 'actor_3_facebook_likes', 'actor_1_facebook_likes', 'gross', 'num_voted_users', 'cast_total_facebook_likes', 'facenumber_in_poster', 'num_user_for_reviews', 'budget', 'title_year', 'actor_2_facebook_likes', 'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'movie_facebook_likes;']


In [18]:
value = 0
df = replace_value(df, columns_to_replace_num, value) 

Теперь значения NaN отсутствуют в датафрейме.

In [19]:
df.isna().sum()

color                        0
director_name                0
num_critic_for_reviews       0
duration                     0
director_facebook_likes      0
actor_3_facebook_likes       0
actor_2_name                 0
actor_1_facebook_likes       0
gross                        0
genres                       0
actor_1_name                 0
movie_title                  0
num_voted_users              0
cast_total_facebook_likes    0
actor_3_name                 0
facenumber_in_poster         0
plot_keywords                0
movie_imdb_link              0
num_user_for_reviews         0
language                     0
country                      0
content_rating               0
budget                       0
title_year                   0
actor_2_facebook_likes       0
imdb_score                   0
aspect_ratio                 0
movie_facebook_likes         0
movie_facebook_likes;        0
dtype: int64

Поскольку явные дубликаты отсутствуют в датафрейме, перейдем к нахождению неявных. Для рассмотрения гипотез необходимо проверить на уникальность значения в Series content_rating.

In [20]:
sorted (df['content_rating'].unique())

['Approved',
 'G',
 'GP',
 'M',
 'NC-17',
 'Not Rated',
 'PG',
 'PG-13',
 'Passed',
 'R',
 'TV-14',
 'TV-G',
 'TV-MA',
 'TV-PG',
 'TV-Y',
 'TV-Y7',
 'Unrated',
 'X',
 'unknown']

Здесь рейтинги 'Not Rated', 'Unrated' являются синонимами определенного ранее 'unknown'. Помимо этого:
'G' = 'Passed';
'PG' = 'GP' = 'M';
'NC-17' = 'X'

In [21]:
def replace_same_ratings (wrong_ratings, correct_rating, df):
    for wrong_rating in wrong_ratings:
        df['content_rating'] = df['content_rating'].replace(wrong_rating, correct_rating)
    return df

In [22]:
df = replace_same_ratings(['Passed', 'Approved'], 'G', df)
df = replace_same_ratings(['GP', 'M'], 'PG', df)
df = replace_same_ratings(['X'], 'NC-17', df)
df = replace_same_ratings(['Not Rated', 'Unrated'], 'unknown', df)

In [23]:
sorted (df['content_rating'].unique())

['G',
 'NC-17',
 'PG',
 'PG-13',
 'R',
 'TV-14',
 'TV-G',
 'TV-MA',
 'TV-PG',
 'TV-Y',
 'TV-Y7',
 'unknown']

Приведем бюджеты и доходы разных годов к значению инфляциии на 2016 год. Для этого пропарсим сайт с необходимой таблицей

In [24]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/'

req = requests.get(url)
data = req.text
soup = BeautifulSoup(data, 'html.parser')

table = soup.find('table')
rows = table.tbody.findAll('tr')

years = []
cpis = []

for row in rows:
    year = row.findAll('td')[0].get_text()
    if year.isdigit() and int(year) < 2017:
        years.append(int(year))
        cpis.append(float(row.findAll('td')[13].get_text()))
        
cpi_table = pd.DataFrame({
    'avg_annual_cpi': cpis,
    'year': years, 
})

cpi_table.head(30)

,avg_annual_cpi,year
0,9.9,1913
1,10.0,1914
2,10.1,1915
3,10.9,1916
4,12.8,1917
5,15.1,1918
6,17.3,1919
7,20.0,1920
8,17.9,1921
9,16.8,1922


Экспортируем таблицу в .csv файл

In [25]:
cpi_table.to_csv ('../data/Lab1_Ziabrew_CPI.csv', index= False )

Запишем функцию перевода валюты

In [26]:
def get_real_value(nominal_amt, old_cpi, new_cpi):
    real_value = (nominal_amt * new_cpi) / old_cpi
    return real_value

Вычислим cpi 2016

In [27]:
CPI_2016 = (cpi_table[cpi_table['year'] == 2016]['avg_annual_cpi'].squeeze())

print(CPI_2016)

240.007


Проверим, в скольких строках датасета неизвестен год выпуска фильма:

In [28]:
print(len(df[df['title_year'] == 0.0]))

107


Поскольку наличие таких строк сильно усложняет работу с реальным бюджетом фильма, с учетом их количества проще от них избавиться.

In [29]:
df = df[df['title_year'] != 0.0]

Вычисление новых сборов и бюджета

In [30]:
real_domestic_gross = []
real_budget_values = []

for index, row in df.iterrows():
    gross =  row['gross']
    budget = row['budget']
    year = row['title_year']
    cpi = cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi'].squeeze()

    real_gross = get_real_value(gross, cpi, CPI_2016)
    real_budget = get_real_value(budget, cpi, CPI_2016)
    real_domestic_gross.append(real_gross)
    real_budget_values.append(real_budget)

df['real_domestic_gross'] = real_domestic_gross
df['real_budget'] = real_budget_values

Вычисление коэффициента прибыльности проката

In [31]:
profits = []
roi_vals = []

for index, row in df.iterrows():
    
    if (row['real_domestic_gross'] == 0 or row['real_budget'] == 0):
        profits.append(0)
        roi_vals.append(0)
        continue
    
    profit = row['real_domestic_gross'] - row['real_budget']
    budget = row['real_budget']
    num = profit - budget
    den = budget
    roi = num/den
    
    profits.append(profit)
    roi_vals.append(roi)

df['profit'] = profits
df['roi'] = roi_vals


Экспортируем обработанный датафрейм в .csv файл

In [32]:
df.to_csv ('../data/Lab1_Ziabrew_MO-221.csv', index= False )

# Гипотеза 1: Окупаемость фильма тем больше, чем меньше возрастной рейтинг фильма.

Избавимся от фильмов с неизвестной окупаемостью

In [33]:
roidf = df[df['roi'] != 0]
roidf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3857 entries, 0 to 62
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      3857 non-null   object 
 1   director_name              3857 non-null   object 
 2   num_critic_for_reviews     3857 non-null   float64
 3   duration                   3857 non-null   float64
 4   director_facebook_likes    3857 non-null   float64
 5   actor_3_facebook_likes     3857 non-null   float64
 6   actor_2_name               3857 non-null   object 
 7   actor_1_facebook_likes     3857 non-null   float64
 8   gross                      3857 non-null   float64
 9   genres                     3857 non-null   object 
 10  actor_1_name               3857 non-null   object 
 11  movie_title                3857 non-null   object 
 12  num_voted_users            3857 non-null   float64
 13  cast_total_facebook_likes  3857 non-null   float64
 14 

Рассмотрим полученный датасет на наличие выбросов

In [34]:
roidf['roi'].describe()

count    3857.000000
mean        4.247754
std       130.169327
min        -1.999982
25%        -1.549085
50%        -0.930105
75%         0.223810
max      7192.485533
Name: roi, dtype: float64

Здесь значение 7192.485533 является выбросом. Воспользуемся методом межквартильного диапазона для очистки newdf от выбросов

In [35]:
Q1 = roidf['roi'].quantile(q=0.25)
Q3 = roidf['roi'].quantile(q=0.75)
IQR = Q3 - Q1
min = Q1-1.5*IQR
max = Q3+1.5*IQR
print(min)
print(max)

roidf_clean = roidf[roidf['roi'] > min]
roidf_clean = roidf[roidf['roi'] < max]

roidf_clean['roi'].describe()

-4.208425785714287
2.88315070952381


count    3479.000000
mean       -0.778273
std         1.081820
min        -1.999982
25%        -1.598834
50%        -1.060347
75%        -0.225392
max         2.877829
Name: roi, dtype: float64

Теперь выведем нужную статистику

In [36]:
roidf_clean.groupby('content_rating')['roi'].mean().sort_values()

content_rating
unknown   -1.668866
R         -0.846855
PG-13     -0.716407
NC-17     -0.607269
PG        -0.594416
G         -0.531637
Name: roi, dtype: float64

Отсюда видно, что со снижением возрастного рейтинга доходность фильмов в целом повышается. Гипотезу можно считать подтвержденной.

# Гипотеза 2: Фильмы с элементами драмы чаще всех имеют IMDB рейтинг выше 8. 

Необходимо понять, какие фильмы включают в себе элементы драмы, а какие нет. Для этого введем датафрейм, равный исходному и отсеим фильмы, рейтинг которых меньше 8

In [37]:
genre_df = df
genre_df = genre_df[genre_df['imdb_score'] >= 8]

Напишем функцию, определяющую наличие нужного признака.

In [38]:
def isdrama(row):
    if row['genres'].find('Drama') != -1:
        return 'drama'
    else:
        return 'not drama'

Введем новый столбец isdrama, определяющий драматический характер фильма

In [39]:
genre_df['isdrama'] = genre_df.apply(isdrama, axis = 1)

C:\Users\ziabr\AppData\Local\Temp\ipykernel_2636\3491964351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_df['isdrama'] = genre_df.apply(isdrama, axis = 1)


Сгруппируем по рейтингу imdb и суммируем по количеству значений

In [40]:
genre_df.groupby('isdrama')['imdb_score'].count()

isdrama
drama        193
not drama     96
Name: imdb_score, dtype: int64

Отсюда явно видно, что драматические фильмы имеют практически двоекратное преимущество среди фильмов, оцененных в 8 и более баллов. Гипотеза подтверждена.

# Гипотеза 3: Фильмы с бюджетом выше среднего (по миру) имеют в среднем более высокую продолжительность, чем остальные.

Введем новый датафрейм

In [41]:
drt_df1 = df

Отсеем фильмы с бюджетом менее медианы 

In [42]:
med = drt_df1['budget'].median()
drt_df1 = drt_df1[drt_df1['budget'] >= med]

Теперь отсеем фильмы с бюджетом более медианы. Для этого введем еще один датафрейм

In [43]:
drt_df2 = df

Аналогично

In [44]:
drt_df2 = drt_df2[drt_df2['budget'] <= med]

Теперь вычислим для каждого датафрейма среднюю продолжительность фильма и выведем частное двух значений

In [45]:
big_budget_dur = drt_df1['duration'].mean()
small_budget_dur = drt_df2['duration'].mean()
print(big_budget_dur/small_budget_dur)

1.1163108932914265


По величине значения можно понять, что какой-то существенной тенденции не наблюдается, поэтому гипотеза опровержена.

# Заключение

Были проверены следующие три гипотезы:

1. Окупаемость фильма тем больше, чем меньше возрастной рейтинг фильма.
2. Фильмы с элементами драмы чаще всех имеют IMDB рейтинг выше 8.
3. Фильмы с бюджетом выше среднего (по миру) имеют в среднем более высокую продолжительность, чем остальные.

Проанализировав данные, мы пришли к выводу:

1. С убывание возрастного рейтинга окупаемость действительно повышается, исключая аномалию в виде PG-13.

*Первая гипотеза* почти полностью подтвердилась.

2. Драм среди фильмов с высоким рейтингом IMDB действительно подавляющее большинство.

*Вторая гипотеза* полностью подтвердилась.

3. Длительность высокобюджетной ленты в целом не отличается от длительности низкобюджетной, поскольку разница на уровне погрешности.
*Третья гипотеза* отвергается. 

# Дополнительное задание

In [ ]:
pvt = pd.pivot_table(df_from_pr, index=["city", "day"], values=['time'], aggfunc= "count")
pvt = pvt.query('day == ["Monday", "Wednesday", "Friday"]')
pvt = pvt.sort_values(by=['time'], ascending=False) 
pvt